In [1]:
import pymysql
import pandas as pd
from surprise import KNNBasic, SVD, SVDpp, NMF, Reader, Dataset
from surprise.model_selection import cross_validate

In [2]:
con = pymysql.connect(host = '127.0.0.1', user = 'root', password = 'kyjin0413!',
                       db = 'lpointsample', charset = 'utf8')
cur = con.cursor(pymysql.cursors.DictCursor)
cur.execute(
    "with T(cust, total) as(select cust,count(*) as total from 제휴사이용정보_view group by cust order  by cust) SELECT j.cust, j.cop_c, round(count(*) /T.total,2) as rating FROM 제휴사이용정보_view j INNER JOIN T t on j.cust = t.cust GROUP BY j.cust, j.cop_c ORDER BY j.cust;")
result=cur.fetchall()
con.commit()
con.close()
df= pd.DataFrame(result)
print(df.head(10))

         cust cop_c rating
0  M000034966   D01   1.00
1  M000059535   C01   1.00
2  M000136117   D01   1.00
3  M000225114   C01   0.10
4  M000225114   C02   0.70
5  M000225114   D02   0.20
6  M000261625   D01   1.00
7  M000419293   C01   0.39
8  M000419293   C02   0.06
9  M000419293   D01   0.33


In [16]:
reader = Reader(rating_scale=(0.1,5))

In [17]:
dataset = Dataset.load_from_df(df[['cust', 'cop_c', 'rating']], reader = reader)

In [13]:
# # KNN을 사용 - 용량 커서 안돌아감
# model = KNNBasic()
# cross_validate(model, dataset, measures = ['rmse', 'mse'], cv = 5, n_jobs = 4, verbose = True)

MemoryError: Unable to allocate 2.89 GiB for an array with shape (19699, 19699) and data type float64

In [18]:
# SVD
model = SVD()
cross_validate(model, dataset, measures = ['rmse', 'mse'], cv = 5, n_jobs = 4, verbose = True)

Evaluating RMSE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4070  0.4056  0.4098  0.4041  0.4198  0.4093  0.0056  
MSE (testset)     0.1657  0.1645  0.1680  0.1633  0.1762  0.1675  0.0046  
Fit time          1.55    1.52    1.52    1.51    1.53    1.53    0.01    
Test time         0.04    0.05    0.05    0.05    0.04    0.05    0.00    


{'test_rmse': array([0.40701987, 0.40559216, 0.4098282 , 0.40406499, 0.41977724]),
 'test_mse': array([0.16566517, 0.164505  , 0.16795915, 0.16326852, 0.17621293]),
 'fit_time': (1.5478625297546387,
  1.5239262580871582,
  1.5199368000030518,
  1.5119585990905762,
  1.5269200801849365),
 'test_time': (0.04488015174865723,
  0.04986691474914551,
  0.04986715316772461,
  0.046875,
  0.040891170501708984)}

In [19]:
# NMF
model = NMF()
cross_validate(model, dataset, measures = ['rmse', 'mse'], cv = 5, n_jobs = 4, verbose = True)

# 0 제외

Evaluating RMSE, MSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3945  0.3966  0.3943  0.3933  0.3961  0.3950  0.0012  
MSE (testset)     0.1557  0.1573  0.1555  0.1547  0.1569  0.1560  0.0009  
Fit time          2.75    2.57    2.64    2.54    2.27    2.56    0.16    
Test time         0.04    0.04    0.05    0.05    0.03    0.04    0.01    


{'test_rmse': array([0.39454412, 0.39655742, 0.39433159, 0.39333933, 0.3961354 ]),
 'test_mse': array([0.15566506, 0.15725779, 0.1554974 , 0.15471583, 0.15692325]),
 'fit_time': (2.7536396980285645,
  2.573122978210449,
  2.639944553375244,
  2.544201135635376,
  2.2669708728790283),
 'test_time': (0.03690147399902344,
  0.03989720344543457,
  0.051863670349121094,
  0.05186033248901367,
  0.03490447998046875)}

In [20]:
# 출력하는데 오래 걸림
model = SVDpp()
cross_validate(model, dataset, measures = ['rmse', 'mse'], cv = 5, n_jobs = 4, verbose = True)

Evaluating RMSE, MSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3256  0.3282  0.3293  0.3239  0.3262  0.3266  0.0019  
MSE (testset)     0.1060  0.1077  0.1085  0.1049  0.1064  0.1067  0.0013  
Fit time          3.02    3.04    3.03    2.99    2.61    2.94    0.17    
Test time         0.11    0.10    0.10    0.12    0.08    0.10    0.01    


{'test_rmse': array([0.32556922, 0.32819389, 0.32932658, 0.32388778, 0.32616811]),
 'test_mse': array([0.10599531, 0.10771123, 0.108456  , 0.10490329, 0.10638564]),
 'fit_time': (3.019930362701416,
  3.0399067401885986,
  3.026926040649414,
  2.987046718597412,
  2.6050376892089844),
 'test_time': (0.10873627662658691,
  0.0997312068939209,
  0.09971809387207031,
  0.11568880081176758,
  0.08178162574768066)}

--------------------------------------------------------------------------------------

In [26]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [28]:
data= pd.DataFrame(result)
data.shape

(43235, 3)

In [29]:
data

,cust,cop_c,rating
0,M000034966,D01,1.00
1,M000059535,C01,1.00
2,M000136117,D01,1.00
3,M000225114,C01,0.10
4,M000225114,C02,0.70
...,...,...,...
43230,M999926092,D02,0.57
43231,M999962961,C01,0.26
43232,M999962961,C02,0.39
43233,M999962961,D01,0.26


In [30]:
ratings = data.pivot_table(index = "cust", columns = "cop_c", values = "rating")
ratings

cop_c,B01,C01,C02,D01,D02,E01
cust,,,,,,
M000034966,NaN,NaN,NaN,1.00,NaN,NaN
M000059535,NaN,1.00,NaN,NaN,NaN,NaN
M000136117,NaN,NaN,NaN,1.00,NaN,NaN
M000225114,NaN,0.10,0.70,NaN,0.20,NaN
M000261625,NaN,NaN,NaN,1.00,NaN,NaN
...,...,...,...,...,...,...
M999708287,NaN,1.00,NaN,NaN,NaN,NaN
M999770689,0.6,NaN,NaN,0.13,0.27,NaN
M999849895,NaN,NaN,NaN,1.00,NaN,NaN


-------------------------------함수 빌드업------------------------------------

In [31]:
u = np.array([np.nan, 4, 3])
print(u)

[nan  4.  3.]


In [32]:
v = np.array([3, 2, 4])
print(v)

[3 2 4]


In [33]:
mask = np.isfinite(u)
mask

array([False,  True,  True])

In [34]:
mask = np.isfinite(u) & np.isfinite(v)
mask

array([False,  True,  True])

In [35]:
u = u[mask]
u

array([4., 3.])

In [36]:
v = v[mask]
v

array([2, 4])

In [39]:
# 분자
uvdot = (u * v).sum()
uvdot

20.0

In [40]:
# 분모
norm1 = (u ** 2).sum()
norm1

25.0

In [41]:
norm2 = (v ** 2).sum()
norm2

20

In [42]:
score = uvdot / np.sqrt(norm1 * norm2)
score

0.8944271909999159

---------------------------함수 제작---------------------------------

In [43]:
def get_cosine_similarity(u, v):
    mask = np.isfinite(u) & np.isfinite(v)
    u = u[mask]
    v = v[mask]

    uvdot = (u * v).sum()

    norm1 = (u ** 2).sum()
    norm2 = (v ** 2).sum()

    score = uvdot / np.sqrt(norm1 * norm2)

    return score

In [50]:
u = ratings.loc['M999926092'] 
v = ratings.loc['M999962961']

get_cosine_similarity(u, v)

0.6581195354686986

In [45]:
from itertools import product

product(ratings.index)

In [46]:
# index_combinations = list(product(ratings.index, repeat = 2))
# index_combinations

MemoryError: 

In [51]:
def get_cosine_similarity_table(rating):
    index_combinations = list(product(ratings.index, repeat = 2))
    index_combinations
    similarity_list = []

    for uname, vname in index_combinations:
        u, v = ratings.loc[uname], ratings.loc[vname]
        score = get_cosine_similarity(u, v)

        similarity = {'u': uname, 'v': vname, 'score': score}

        similarity_list.append(similarity)
    similarity_list = DataFrame(similarity_list)
    similarity_table = pd.pivot_table(similarity_list, index = 'u', columns = 'v', values = 'score')

    return similarity_table

similarity_table = get_cosine_similarity_table(ratings)
similarity_table

MemoryError: 

In [ ]:
def predict_ratings(cust_name, cop_c):

    neighbors_ratings = ratings[cop_c].drop(index = 'cust_name')

    

----------------------------------------------------------

In [54]:
ratings.index

Index(['M000034966', 'M000059535', 'M000136117', 'M000225114', 'M000261625',
       'M000419293', 'M000494848', 'M000504230', 'M000557840', 'M000608261',
       ...
       'M999468256', 'M999492154', 'M999493501', 'M999515910', 'M999599111',
       'M999708287', 'M999770689', 'M999849895', 'M999926092', 'M999962961'],
      dtype='object', name='cust', length=21769)

In [55]:
ratings.columns

Index(['B01', 'C01', 'C02', 'D01', 'D02', 'E01'], dtype='object', name='cop_c')

In [ ]:
ratings_combinations = list(product(ratings.index, ratings.columns))
rating_list = []

for cust_name, cop_c in ratings_combinations:
    score = predict_ratings(cust_name, cop_c)

    rating_predict = {'cust': cust_name, 'cop_c': cop_c, 'score': score}

    rating_list.append(rating_predict)

rating_list = DataFrame(rating_list)
rating_table = pd.pivot_table(rating_list, index = "cust", columns = 'cop_c', values = 'score')
rating_table